# HMP an C3D features - Human/object motion proposition
 Mount Google drive to Read Files

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir("/content/drive/My Drive/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


### Method to calcualte spearman's Correlation Coefficient

In [0]:
def Get_score(Y_pred,Y_true):
    '''Calculate the Spearmann"s correlation coefficient'''
    Y_pred = np.squeeze(Y_pred)
    Y_true = np.squeeze(Y_true)
    if Y_pred.shape != Y_true.shape:
        print('Input shapes don\'t match!')
    else:
        if len(Y_pred.shape) == 1:
            Res = pd.DataFrame({'Y_true':Y_true,'Y_pred':Y_pred})
            score_mat = Res[['Y_true','Y_pred']].corr(method='spearman',min_periods=1)
            print('The Spearman\'s correlation coefficient is: %.3f' % score_mat.iloc[1][0])
        else:
            for ii in range(Y_pred.shape[1]):
                Get_score(Y_pred[:,ii],Y_true[:,ii])


### Import all necessary libraries

In [0]:
import numpy as np
import pandas as pd
from string import punctuation
from collections import Counter
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import itertools 
import io
from sklearn.metrics import mean_squared_error
from tensorflow.python.keras import Sequential
from tensorflow.python.keras import layers
from tensorflow.python.keras import regularizers
from tensorflow.python.keras.preprocessing.text import Tokenizer
from keras import optimizers

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

Read the CSV from Drive - Dependant Variables

In [0]:
label_path = './Assignment/Dev-set/Ground-truth/'
labels=pd.read_csv(label_path+'ground-truth.csv')

### To Read HMP features from Drive and store it in local drive

In [0]:
''' Method to read HMP'''
def read_HMP(fpath, fname, HMP_map):
    with open(fname) as f:
        for line in f:
            pairs=line.split()
            HMP_temp = { int(p.split(':')[0]) : float(p.split(':')[1]) for p in pairs}
    # there are 6075 bins, fill zeros
    HMP = np.zeros(6075)
    for idx in HMP_temp.keys():
        HMP[idx-1] = HMP_temp[idx]            
    return HMP

In [0]:
ind = 0
video_list = []
for i in range(1, 7494):
  name = 'video' + str(i) 
  video_list.append(name)

In [0]:
!apt install unzip
!unzip "./HMPReal.zip" -d ""

In [0]:
HMP_map = {}
hmp_arr = np.empty((0,6075), float)
# load the captions
for filename in video_list:
  filename = filename + '.txt'
  file_path = './HMP/' + filename
  
  if(os.path.exists(file_path)):
    HMP = read_HMP(file_path, file_path, HMP_map)
    hmp_arr= np.vstack([hmp_arr, HMP])

In [0]:
column_list = []
for i in range(1, 6076):
  cname = 'hmp' + str(i)
  column_list.append(cname)

In [10]:
len(column_list)

6075

In [0]:
hmp_df = pd.DataFrame(hmp_arr, columns=column_list)


In [0]:
hmp_df['video_name'] = aes_mean_df[['name']]

In [0]:
cols = hmp_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
hmp_df = hmp_df[cols]
hm_df_algo = hmp_df.iloc[:, 1:]

In [0]:
hmp_df.to_csv('/content/drive/My Drive/HMP.csv')

### Load HMP from local drive to perfrom Regression

In [39]:
hmp_df = pd.read_csv('HMP.csv')
hmp_df = hmp_df.iloc[: , 2:]
hmp_df = hmp_df.astype(np.float64)
hmp_df.head()

,hmp1,hmp2,hmp3,hmp4,hmp5,hmp6,hmp7,hmp8,hmp9,hmp10,hmp11,hmp12,hmp13,hmp14,hmp15,hmp16,hmp17,hmp18,hmp19,hmp20,hmp21,hmp22,hmp23,hmp24,hmp25,hmp26,hmp27,hmp28,hmp29,hmp30,hmp31,hmp32,hmp33,hmp34,hmp35,hmp36,hmp37,hmp38,hmp39,hmp40,...,hmp6036,hmp6037,hmp6038,hmp6039,hmp6040,hmp6041,hmp6042,hmp6043,hmp6044,hmp6045,hmp6046,hmp6047,hmp6048,hmp6049,hmp6050,hmp6051,hmp6052,hmp6053,hmp6054,hmp6055,hmp6056,hmp6057,hmp6058,hmp6059,hmp6060,hmp6061,hmp6062,hmp6063,hmp6064,hmp6065,hmp6066,hmp6067,hmp6068,hmp6069,hmp6070,hmp6071,hmp6072,hmp6073,hmp6074,hmp6075
0,0.125563,0.024036,0.000314,0.0,0.015864,0.000358,0.0,0.000000,0.000086,0.0,0.0,0.0,0.0,0.002795,0.000054,0.0,0.000000,0.000037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000002,0.000173,0.000459,0.000000,0.000148,0.000104,0.000000,0.000121,0.000551,0.000000,0.000114,0.000884,0.000002,0.000116,0.000077,0.000002,0.000027,0.000136,0.000000,0.000000,0.000002,0.000000,0.000091,0.000035,0.000000,0.000163,0.000467,0.000002,0.000010,0.000017,0.000000,0.000393,0.000279,0.000000,0.000289,0.001926,0.000000,0.000086,0.000580,0.000000
1,0.007526,0.001421,0.000068,0.0,0.001184,0.000143,0.0,0.000000,0.000079,0.0,0.0,0.0,0.0,0.000246,0.000024,0.0,0.000000,0.000042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000685,0.000308,0.001054,0.000751,0.000176,0.000062,0.000000,0.000123,0.000398,0.000086,0.000246,0.000433,0.000446,0.000143,0.000053,0.000000,0.000053,0.000099,0.000009,0.000004,0.000033,0.000004,0.000051,0.000035,0.000000,0.000062,0.000358,0.000035,0.000024,0.000083,0.000053,0.000244,0.000066,0.000000,0.000081,0.000617,0.000094,0.000220,0.000762,0.001224
2,0.109584,0.018978,0.000289,0.0,0.008774,0.000208,0.0,0.000002,0.000088,0.0,0.0,0.0,0.0,0.002046,0.000061,0.0,0.000000,0.000038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000083,0.000057,0.000158,0.000073,0.000021,0.000009,0.000002,0.000019,0.000095,0.000021,0.000019,0.000090,0.000073,0.000050,0.000024,0.000000,0.000012,0.000021,0.000000,0.000000,0.000002,0.000000,0.000017,0.000007,0.000002,0.000066,0.000203,0.000026,0.000002,0.000040,0.000007,0.000054,0.000045,0.000000,0.000028,0.000291,0.000033,0.000052,0.000258,0.000215
3,0.120431,0.013561,0.000277,0.0,0.018974,0.000913,0.0,0.000024,0.000713,0.0,0.0,0.0,0.0,0.002496,0.000149,0.0,0.000011,0.000157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000210,0.000434,0.000543,0.000412,0.000412,0.000045,0.000003,0.000144,0.000282,0.000037,0.000197,0.000218,0.000157,0.000237,0.000021,0.000000,0.000040,0.000056,0.000008,0.000005,0.000013,0.000019,0.000168,0.000013,0.000000,0.000133,0.000202,0.000029,0.000029,0.000035,0.000059,0.001110,0.000075,0.000008,0.000333,0.000793,0.000101,0.000588,0.000503,0.000452
4,0.005026,0.001356,0.000055,0.0,0.000665,0.000029,0.0,0.000000,0.000024,0.0,0.0,0.0,0.0,0.000147,0.000020,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000160,0.000996,0.001604,0.000103,0.000768,0.000215,0.000009,0.000415,0.000926,0.000020,0.000538,0.001178,0.000050,0.000518,0.000169,0.000007,0.000134,0.000169,0.000007,0.000026,0.000046,0.000007,0.000373,0.000088,0.000000,0.000338,0.000441,0.000029,0.000070,0.000149,0.000009,0.000882,0.000200,0.000009,0.000559,0.001097,0.000018,0.000632,0.001128,0.000064


In [0]:
hmp_array = hmp_df.values

In [0]:
X = hmp_array

Y = labels[['short-term_memorability']].values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42)

In [14]:
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(4800, 6075) (1200, 6075) (4800, 1) (1200, 1)


### Video Memorability - HMP Randomo Forest Regressor

Short term scores



In [42]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(n_estimators=1000, max_features= 5, min_samples_leaf = 4, min_samples_split= 5)
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
Get_score(y_pred, Y_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


The Spearman's correlation coefficient is: 0.302


MSE : 0.005596628843497854

In [45]:
from sklearn.metrics import mean_squared_error
mean_squared_error(Y_test, y_pred) 

0.005596628843497854

Long term Scores

In [0]:
X = hmp_array

Y = labels[['long-term_memorability']].values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42)

In [47]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(n_estimators=1000, max_features= 5, min_samples_leaf = 4, min_samples_split= 5)
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
Get_score(y_pred, Y_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


The Spearman's correlation coefficient is: 0.118


In [48]:
from sklearn.metrics import mean_squared_error
mean_squared_error(Y_test, y_pred) 

0.021530176798856833

### Perfomr PCA to reduce dimensions

In [19]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_PCA_train)
X_PCA_train = scaler.transform(X_PCA_train)
X_PCA_test = scaler.transform(X_PCA_test)
from sklearn.decomposition import PCA
pca = PCA(.95)
pca.fit(X_PCA_train)
pca.n_components_

1360

In [0]:
X_PCA_train, X_PCA_test, Y_PCA_train, Y_PCA_test = train_test_split(hmp_array, Y,  test_size= 0.2, random_state=42)

In [0]:
X_PCA_train = pca.transform(X_PCA_train)
X_PCA_test = pca.transform(X_PCA_test)

In [21]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(n_estimators=1000, max_features= 5, min_samples_leaf = 3, min_samples_split= 8)
clf.fit(X_PCA_train, Y_train)
y_pred = clf.predict(X_PCA_test)
Get_score(y_pred, Y_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


The Spearman's correlation coefficient is: 0.070


### Read C3D from shared drive and save it in local drive

In [0]:
''' Method to read c3d'''
def read_c3d(fpath, fname, c3d_map):
  with open(fpath) as f:
    value_list = []
    for line in f:
      pairs=line.split(' ')
      for value in pairs:
        value_list.append(value)
    c3d_map[fname] = value_list

In [0]:
video_list = []
for i in range(1, 7494):
  name = 'video' + str(i)
  filename = name + '.txt'
  file_path = './Assignment/Dev-set/C3D/' + filename
  if(os.path.exists(file_path)):
    video_list.append(name)

In [0]:
c3d_map = {}
#hmp_arr = np.empty((0,6075), float)
# load the captions
for filename in video_list:
  filename = filename + '.txt'
  file_path = './Assignment/Dev-set/C3D/' + filename
  read_c3d(file_path, filename, c3d_map)

In [0]:
c3d_df = pd.DataFrame.from_dict(c3d_map,  orient='index')

In [0]:
c3d_df.head()

In [0]:
c3d_df.to_csv('/content/drive/My Drive/c3d_df.csv')

### **C3D - Train models for video memorability**

In [0]:
c3d_df = pd.read_csv('c3d_df.csv')

In [0]:
c3d_df = c3d_df.iloc[:, 1:-1]

In [0]:
c3d_df.head()

In [0]:
c3d_arr  = c3d_df.values

### Video Memorability - C3D

Short-term memorability Score




In [0]:
X = c3d_arr
#Y = labels[['short-term_memorability','long-term_memorability']].values
Y = labels[['short-term_memorability']].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [53]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(n_estimators = 2000, max_features= 4, min_samples_leaf = 5, min_samples_split= 8)
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
Get_score(y_pred, Y_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


The Spearman's correlation coefficient is: 0.332


In [54]:
from sklearn.metrics import mean_squared_error
mean_squared_error(Y_test, y_pred) 

0.005469710068597566

Long-term memorability score

In [0]:
0.021530176798856833

In [0]:
X = c3d_arr
#Y = labels[['short-term_memorability','long-term_memorability']].values
Y = labels[['long-term_memorability']].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [56]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(n_estimators = 2000, max_features= 4, min_samples_leaf = 5, min_samples_split= 8)
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
Get_score(y_pred, Y_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


The Spearman's correlation coefficient is: 0.145


In [57]:
from sklearn.metrics import mean_squared_error
mean_squared_error(Y_test, y_pred) 

0.021379138059125096

### C3D + HMP - video memorability

In [0]:
hmp_c3d = np.concatenate((c3d_arr, hmp_array), axis=1)

In [0]:
X = hmp_c3d
#Y = labels[['short-term_memorability','long-term_memorability']].values
Y = labels[['long-term_memorability']].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [60]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(n_estimators = 2000, max_features= 4, min_samples_leaf = 5, min_samples_split= 8)
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
Get_score(y_pred, Y_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


The Spearman's correlation coefficient is: 0.130


In [61]:
mean_squared_error(Y_test, y_pred) 

0.021457148513859534

In [0]:
X = hmp_c3d
#Y = labels[['short-term_memorability','long-term_memorability']].values
Y = labels[['short-term_memorability']].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [63]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(n_estimators = 2000, max_features= 4, min_samples_leaf = 5, min_samples_split= 8)
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
Get_score(y_pred, Y_test)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


The Spearman's correlation coefficient is: 0.308


In [64]:
mean_squared_error(Y_test, y_pred) 

0.005593858374323793